In [16]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import torch


### Load dataset - Preprocessing
X, y = load_digits().data, load_digits().target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                   test_size=0.2,
                                                   random_state=15)


### Build network
IN_SIZE = 64
HIDDEN_SIZE = 50
OUT_SIZE = 10
LR=0.0005

class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(IN_SIZE , HIDDEN_SIZE)
        self.l2 = torch.nn.Linear(HIDDEN_SIZE, OUT_SIZE)

    def forward(self, x):
        x = torch.relu(self.l1(x))
        y_logits = self.l2(x)
        return y_logits

net = Net()
criterion = torch.nn.CrossEntropyLoss(reduction='sum')
opti = torch.optim.SGD(net.parameters(), lr=LR)

### Training
NEPOCHS = 50
BATCH_SIZE = 32

for epoch in range(NEPOCHS):

    p = np.random.permutation(len(X_train))
    X_train, y_train = X_train[p], y_train[p]
    for ki in range(0, len(X_train), BATCH_SIZE):
        X_batch = torch.from_numpy(X_train[ki:ki+BATCH_SIZE])
        y_batch = torch.from_numpy(y_train[ki:ki+BATCH_SIZE])
    
        net.zero_grad()
        y_logits = net(X_batch)
        loss = criterion(y_logits, y_batch)
        loss.backward()
        opti.step()
        
    X = torch.from_numpy(X_train)
    y = torch.from_numpy(y_train)
    y_logits = net(X)
    loss = criterion(y_logits, y)
    preds = torch.argmax(y_logits, dim=1)
    acc = y.eq(preds).sum().float() / len(X)
    print('Epoch {}: Loss = {}, Accuracy = {}'.format(epoch+1, 
                                                      loss.data,
                                                     acc))
    
    
### Evaluate
X = torch.from_numpy(X_test)
y = torch.from_numpy(y_test)
preds = torch.argmax(net(X), dim=1)
acc = y.eq(preds).sum().float() / len(X)
print('Test Accuracy = {}'.format(acc))

X0 = X[0:1]
y0 = y[0]
probs = torch.softmax(net(X0), dim=1)
print('proba:', probs.data.numpy())
print('true class:', y0)

Epoch 1: Loss = 553.462890625, Accuracy = 0.9095337390899658
Epoch 2: Loss = 315.73345947265625, Accuracy = 0.9464161396026611
Epoch 3: Loss = 235.0081024169922, Accuracy = 0.9589422345161438
Epoch 4: Loss = 239.98345947265625, Accuracy = 0.9547668695449829
Epoch 5: Loss = 163.33432006835938, Accuracy = 0.9735560417175293
Epoch 6: Loss = 144.26361083984375, Accuracy = 0.9812108278274536
Epoch 7: Loss = 143.83868408203125, Accuracy = 0.9805149435997009
Epoch 8: Loss = 126.81071472167969, Accuracy = 0.9819067716598511
Epoch 9: Loss = 137.13677978515625, Accuracy = 0.9749478101730347
Epoch 10: Loss = 103.44094848632812, Accuracy = 0.9839944243431091
Epoch 11: Loss = 81.99950408935547, Accuracy = 0.9930410385131836
Epoch 12: Loss = 79.69145202636719, Accuracy = 0.9909533858299255
Epoch 13: Loss = 72.37704467773438, Accuracy = 0.9916492700576782
Epoch 14: Loss = 65.76701354980469, Accuracy = 0.9937369227409363
Epoch 15: Loss = 73.16390991210938, Accuracy = 0.9916492700576782
Epoch 16: Loss 